This notebook will plot the biological samples with the nearest boat and sailbuoy acoustics.

In [2]:
import glob
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cartopy.crs as ccrs
import cartopy
import sb_tools
import seaborn as sns

from math import radians, cos, sin, asin, sqrt
from pyechoplot.plotting import plot_Sv, plot_mask, save_png_plot, setup_ek500_cmap


In [3]:
sns.set_theme(style="darkgrid")
sns.color_palette("viridis", as_cmap=True)
sns.set(rc={'figure.figsize':(11.7,8.27)})
title_font = {
    'fontname': 'DejaVu Sans', 'size': '15', 'color': 'black',
    'weight': 'medium'
}
axis_font = {'fontname': 'DejaVu Sans', 'size': '15', 'color': 'black'}

# Extract time and positions closest to stations
For boat acoustics and SB
Stations of interest in Tromsøflaket are Station 7, 9, 11, 15, 16 and 17.

In [4]:
SP_stations_df = pd.read_excel('C:/Users/mbd/OneDrive - Akvaplan-niva AS/PhD-APN/ChaptersandExperiments/Tromsøflaket/SeaPatchesStations.xlsx',header=0)
SP_lons = SP_stations_df['longitude (deg)']
SP_lats = SP_stations_df['latitude (deg)']
stn_num = SP_stations_df['Station']

In [5]:
#load Sailbuoy position
SB_gps_df = pd.read_excel('C:/Users/mbd/OneDrive - Akvaplan-niva AS/PhD-APN/ChaptersandExperiments/Tromsøflaket/SB_env_tromsøflaket2018.xlsx',header=0, engine="openpyxl")

#load Sea patches cruise location
SP_gps_df_20 = pd.read_csv('C:/Users/mbd/OneDrive - Akvaplan-niva AS/PhD-APN/ChaptersandExperiments/Tromsøflaket/SP_GPSExport_20.gps.csv', usecols=[1,2,4,5])
SP_gps_df = pd.read_csv('C:/Users/mbd/OneDrive - Akvaplan-niva AS/PhD-APN/ChaptersandExperiments/Tromsøflaket/SP_GPSExport.gps.csv', usecols=[1,2,4,5])
SP_gps_df = pd.concat([SP_gps_df_20, SP_gps_df], ignore_index=True)
# Add datetime
SP_gps_df['GPS_datetime']=pd.to_datetime(SP_gps_df['GPS_date']+' '+SP_gps_df['GPS_time'])
SP_gps_df


,GPS_date,GPS_time,Latitude,Longitude,GPS_datetime
0,2018-06-20,17:55:02,69.691767,16.675131,2018-06-20 17:55:02
1,2018-06-20,17:55:02,69.691767,16.675131,2018-06-20 17:55:02
2,2018-06-20,17:55:02,69.691767,16.675131,2018-06-20 17:55:02
3,2018-06-20,17:55:03,69.691780,16.675205,2018-06-20 17:55:03
4,2018-06-20,17:55:03,69.691780,16.675205,2018-06-20 17:55:03
...,...,...,...,...,...
3389536,2018-06-27,06:50:17,69.800275,19.384785,2018-06-27 06:50:17
3389537,2018-06-27,06:50:17,69.800275,19.384785,2018-06-27 06:50:17
3389538,2018-06-27,06:50:17,69.800270,19.384726,2018-06-27 06:50:17
3389539,2018-06-27,06:50:17,69.800270,19.384726,2018-06-27 06:50:17


In [6]:
def distance(s_lat, s_lng, e_lat, e_lng):

   # approximate radius of earth in km
   R = 6373.0

   s_lat = s_lat*np.pi/180.0                      
   s_lng = np.deg2rad(s_lng)     
   e_lat = np.deg2rad(e_lat)                       
   e_lng = np.deg2rad(e_lng)  

   d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat)*np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2

   return 2 * R * np.arcsin(np.sqrt(d))

In [7]:
flaket_stations = np.array([7,9,11,15,16,17])
SB_closest_index = np.zeros(len(flaket_stations))
SP_closest_index = np.zeros(len(flaket_stations))

for ind in range(len(flaket_stations)):
    station_index = flaket_stations[ind]-1
    
    # Nearest Sailbuoy location to station
    dist = distance(SP_lats[station_index],SP_lons[station_index], SB_gps_df['latitude (deg)'], SB_gps_df['longitude (deg)'])
    SB_closest_index[ind] = np.argmin(dist)
    
    #Nearest sea patches cruise to station
    dist_SP = distance(SP_lats[station_index],SP_lons[station_index], SP_gps_df['Latitude'], SP_gps_df['Longitude'])
    SP_closest_index[ind] = np.argmin(dist_SP)

# When was it closest?

In [8]:
SB_gps_df['datetime'][SB_closest_index]

3.0     2018-06-21 09:40:00
52.0    2018-06-25 13:30:00
34.0    2018-06-22 11:00:00
85.0    2018-06-26 22:30:00
127.0   2018-06-29 22:00:00
119.0   2018-06-29 06:00:00
Name: datetime, dtype: datetime64[ns]

In [9]:
SP_gps_df['GPS_datetime'][SP_closest_index]

215955.0    2018-06-21 03:54:54
1018320.0   2018-06-22 17:03:40
1297692.0   2018-06-23 05:59:40
1924278.0   2018-06-24 11:00:10
2326203.0   2018-06-25 05:36:37
2630523.0   2018-06-25 19:41:56
Name: GPS_datetime, dtype: datetime64[ns]

# Open and extract Sv 
From boat and SB acoustics. Using code from SSLEM toolbox, Proud et al. 2015

In [10]:
# Time nearest to Station 7
stn_ind = SB_closest_index[flaket_stations == 7]
print(SB_gps_df['datetime'][stn_ind])

SP_stn_ind = SP_closest_index[flaket_stations == 7]
print(SP_gps_df['GPS_datetime'][SP_stn_ind])


3.0   2018-06-21 09:40:00
Name: datetime, dtype: datetime64[ns]
215955.0   2018-06-21 03:54:54
Name: GPS_datetime, dtype: datetime64[ns]


In [11]:
fname = 'D:/GLIDER2018/Echosounder/EVexport/boat_20180622PM.xlsx'
# Open excel file into dataframe
Sv_df = pd.read_excel(fname, header=0)
## Put the mean Sv data into a table, with x axis as pings and y axis as layer depth min (for now)
Sv_table = Sv_df.pivot(index='Layer_depth_min', columns='Ping_S', values='Sv_mean')
# Convert dataframe to numpy array to comply with the algorithm
Sv = np.array(Sv_table)

fname_SB = 'D:/GLIDER2018/Echosounder/EVexport/SBanalysis_20180621_24.csv'
# Open excel file into dataframe
SB_df = pd.read_excel(fname_SB, header=0)
## Put the mean Sv data into a table, with x axis as pings and y axis as layer depth min (for now)
Sv_table_SB = SB_df.pivot(index='Layer_depth_min', columns='Ping_S', values='Sv_mean')
# Convert dataframe to numpy array to comply with the algorithm
Sv_SB = np.array(Sv_table_SB)


plt.figure(figsize=(15,10))
plt.subplot(3,1,1)
plot_Sv(Sv_SB)
plt.subplot(3,1,2)
plot_Sv(Sv)
plt.title('Findings at Station 7')

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbfProce'

In [ ]:
# Time nearest to Station 9
stn_ind = closest_index[flaket_stations == 9]
print(SB_env_df['datetime'][stn_ind])

SP_stn_ind = SP_closest_index[flaket_stations == 9]
print(SP_env_df['GPS_datetime'][SP_stn_ind])


In [ ]:
fname = 'D:/GLIDER2018/Echosounder/EVexport/boat_20180625PM.xlsx'
# Open excel file into dataframe
Sv_df = pd.read_excel(fname, header=0)
## Put the mean Sv data into a table, with x axis as pings and y axis as layer depth min (for now)
Sv_table = Sv_df.pivot(index='Layer_depth_min', columns='Ping_S', values='Sv_mean')
# Convert dataframe to numpy array to comply with the algorithm
Sv = np.array(Sv_table)

fname_SB = 'D:/GLIDER2018/Echosounder/EVexport/SBanalysis_20180625pm.xlsx'
# Open excel file into dataframe
SB_df = pd.read_excel(fname_SB, header=0)
## Put the mean Sv data into a table, with x axis as pings and y axis as layer depth min (for now)
Sv_table_SB = SB_df.pivot(index='Layer_depth_min', columns='Ping_S', values='Sv_mean')
# Convert dataframe to numpy array to comply with the algorithm
Sv_SB = np.array(Sv_table_SB)


plt.figure(figsize=(15,10))
plt.subplot(3,1,1)
plot_Sv(Sv)
plt.subplot(3,1,2)
plot_Sv(Sv_SB)
plt.title('Findings at Station 9')

In [ ]:
# Time nearest to Station 11

SP_stn_ind = SP_closest_index[flaket_stations == 16]
print(SP_env_df['GPS_datetime'][SP_stn_ind])

stn_ind = closest_index[flaket_stations == 16]
print(SB_env_df['datetime'][stn_ind])


In [ ]:
fname = 'D:/GLIDER2018/Echosounder/EVexport/boat_20180625AM.xlsx'
# Open excel file into dataframe
Sv_df = pd.read_excel(fname, header=0)
## Put the mean Sv data into a table, with x axis as pings and y axis as layer depth min (for now)
Sv_table = Sv_df.pivot(index='Layer_depth_min', columns='Ping_S', values='Sv_mean')
# Convert dataframe to numpy array to comply with the algorithm
Sv = np.array(Sv_table)

fname_SB = 'D:/GLIDER2018/Echosounder/EVexport/SBanalysis_20180629pm.xlsx'
# Open excel file into dataframe
SB_df = pd.read_excel(fname_SB, header=0)
## Put the mean Sv data into a table, with x axis as pings and y axis as layer depth min (for now)
Sv_table_SB = SB_df.pivot(index='Layer_depth_min', columns='Ping_S', values='Sv_mean')
# Convert dataframe to numpy array to comply with the algorithm
Sv_SB = np.array(Sv_table_SB)


plt.figure(figsize=(15,10))
plt.subplot(3,1,1)
plt.imshow(Sv)
plt.subplot(3,1,2)
plot_Sv(Sv_SB)
plt.title('Findings at Station 9')

In [ ]:
Date= pd.to_datetime(Sv_df.Date_M, format='%Y%m%d')
Datetime = pd.to_datetime(Date.dt.strftime('%Y%m%d')+ " "+ Sv_df.Time_M.astype('str'))

fig = plt.figure()
c = plt.pcolor(np.unique(Datetime),np.unique(Sv_df['Layer_depth_min']),Sv, cmap="viridis", vmin=-84, vmax=-34)
fig.colorbar(c)

In [ ]:
np.shape(Sv)

In [ ]:
np.shape(np.unique(Sv_df['Layer_depth_min']))

In [ ]:
np.shape(np.unique(Sv_df['Date_M','Time_M']))

In [ ]:
pd.to_datetime(Sv_df['Date_M']+' '+Sv_df['Time_M'])

In [ ]:
Date= pd.to_datetime(Sv_df.Date_M, format='%Y%m%d')
Datetime = pd.to_datetime(Date.dt.strftime('%Y%m%d')+ " "+ Sv_df.Time_M.astype('str'))

In [ ]:
np.unique(Datetime)